<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/extract/gradioprompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install easyocr gradio pythainlp langchain langchain_huggingface langchain_community pytesseract transformers
!sudo apt-get install ghostscript

Processing triggers for man-db (2.10.2-1) ...


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

resume_directory = '/content/drive/MyDrive/AIEngineer/resume/resume_LLM'
pdf_files = [os.path.join(resume_directory, f) for f in os.listdir(resume_directory) if f.endswith('.pdf')]

# Print the list of PDF files to verify
print(pdf_files)

['/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume BA 2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume BA 3.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume ba 5.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume ba2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume baa.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 6.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 7.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 3.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 4.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 5.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 1.pdf']


## **Job Qualification**

| **ᴊᴏʙ** | **ᴘᴏꜱɪᴛɪᴏɴ** | **ᴄᴏᴜɴᴛ** |
|:---:|:---:|:---:|
| 1 | ʙᴜꜱɪɴᴇꜱꜱ ᴀɴᴀʟʏꜱᴛ | 5 |
| 2 | ᴀɪ ᴇɴɢɪɴᴇᴇʀ | 7 |


In [4]:
import os
import re
import numpy as np
import pandas as pd
import easyocr
import subprocess
from pythainlp.phayathaibert.core import NamedEntityTagger
import gradio as gr
from PIL import Image
import requests
from threading import Semaphore
from concurrent.futures import ThreadPoolExecutor, as_completed
from huggingface_hub import InferenceClient
import pytesseract
from nltk.tokenize import sent_tokenize

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.26M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.4M [00:00<?, ?B/s]

In [11]:
resume_prompt = """
# Resume Evaluation Prompt

**You are a career coach and resume evaluator**, tasked with thoughtfully assessing a candidate's resume based on a specific job description. Your goal is to strike a balance between the candidate’s qualifications and their potential for growth. Keep in mind that real-world hiring involves not only meeting immediate requirements but also identifying long-term potential and adaptability.

## Instructions:
- **Focus primarily on the job description**, but also consider any unique skills and experience that the candidate offers.
- Your evaluation should prioritize candidates who align well with the role but remain open to those who might provide unexpected strengths.
- **Don’t rely solely on a perfect match**—instead, take a holistic view of the candidate to assess their overall fit, potential, and growth trajectory.

### Evaluation Criteria:

1. **Strong Fit (PASS)**:
   - The candidate aligns well with the job requirements and demonstrates strong potential to excel, even if some qualifications are missing.
   - Consider their **unique strengths**, **relevant experience**, and **transferable skills**.
   - If they show adaptability and a strong overall fit, classify them as "Strong Fit."

2. **Promising (RECONSIDER)**:
   - The candidate aligns with some aspects of the job but has notable gaps.
   - However, their experience suggests they could grow into the role.
   - Consider them as a candidate with potential, particularly if their overall background suggests they would be worth interviewing or reviewing further.

3. **Not a Fit (UNRELATED)**:
   - The candidate’s background does not align with the job description and there is no significant potential for the role.
   - Their experience, while potentially strong in other areas, is unrelated to the requirements for this position.

## Evaluation Process:

1. **Step 1**: Carefully review the **job description** and extract the core requirements and preferred qualifications.
2. **Step 2**: Compare the **resume** to the job description:
   - Identify relevant qualifications and skills.
   - Note any **transferable skills** or **unique strengths** that the candidate brings.
3. **Step 3**: Consider the candidate's **overall potential**, adaptability, and long-term growth.
4. **Step 4**: Classify the resume and provide clear **feedback** on how well the candidate aligns with the job description, highlighting their strengths and areas for improvement.

### What Not to Do:
- **Don’t disqualify candidates** for not meeting every single requirement. Assess their ability to adapt and grow.
- **Don’t ignore transferable skills** that could be useful in the role.
- **Remain open-minded** to candidates who may bring unique value, even if their experience isn’t a perfect match.

## Selection Criteria:

- **Relevance to Job Requirements**: Focus on how well the candidate’s experience and skills align with both the required and preferred qualifications of the job description.
- **Transferable Skills**: Identify skills and experiences that, while not directly related, could be beneficial in the role.
- **Potential**: Look beyond immediate requirements to assess whether the candidate shows promise to grow into the role and contribute long-term.
- **Holistic View**: Evaluate the resume with a broader perspective, considering adaptability, cultural fit, and overall value to the team or company.

---

### **Few-Shot Examples** (for internal reference, never copy them)

#### **Job Description**:
- Required: 5+ years of experience in software development, proficiency in Python and JavaScript, experience with cloud platforms, and leadership skills. Bachelor's degree in Computer Science or related field.
- Preferred: Master's degree, DevOps experience, and familiarity with modern software development practices.

#### **Resume**:
- Candidate has 6 years of software development experience, proficient in Python and JavaScript, and cloud platform experience. Holds a Master’s degree in Computer Science and has led teams of up to 10 people. Some exposure to DevOps.

#### **Evaluation**:
- **Classification**: Pass
- **Rationale**: The candidate meets the core requirements and brings valuable experience in leadership and DevOps. They have demonstrated the ability to succeed in a complex environment, making them a strong fit for the role.

---

#### **Job Description**:
- Required: 3+ years of software development experience, proficiency in Python and JavaScript, experience with cloud platforms. Bachelor’s degree required.
- Preferred: Experience with DevOps and React.js.

#### **Resume**:
- The candidate has 3 years of experience in automation and Python development. They have worked on cloud-based projects but have less direct experience with JavaScript or React.js.

#### **Evaluation**:
- **Classification**: Reconsider
- **Rationale**: The candidate shows strong Python and cloud platform experience but lacks direct JavaScript experience. Given their expertise in automation and cloud projects, they might still be a good fit with some support in JavaScript.

---

#### **Job Description**:
- Required: 3+ years of software development experience, proficiency in Python and JavaScript, experience with cloud platforms.
- Preferred: DevOps knowledge and experience in React.js.

#### **Resume**:
- The candidate has 2 years of experience as a graphic designer and social media manager, with no direct experience in software development or related fields.

#### **Evaluation**:
- **Classification**: Unrelated
- **Rationale**: The candidate’s background is not relevant to the software development role. Their experience in design and social media does not align with the technical requirements of the position.

---

- **Job Description**:
{jobdescribe} Summarize the key required and preferred qualifications of the job description.
- **Resume**:
{resume} Summarize the relevant skills and experience of the candidate.

## **Template for Evaluation**:
**Classification**: (Pass / Reconsider / Unrelated)
**Rationale**: Provide a clear and concise explanation for your classification. Focus on how well the candidate aligns with the job description, highlight any transferable skills or potential, and explain any significant gaps. Mention both strengths and areas for improvement.
"""

In [13]:
# Define the model parameters
model_params = {
    "max_new_tokens": 1500,
    "temperature": 0.1,
    "top_p": 0.95,
    "repetition_penalty": 1.0
}
semaphore = Semaphore(50)

def convert_pdf_to_images(pdf_file, output_folder="/content/images", dpi=300):
    try:
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        output_format = os.path.join(output_folder, "page_%03d.png")
        gs_command = [
            "gs",
            "-sDEVICE=png16m",
            f"-r{dpi}",
            "-o", output_format,
            pdf_file
        ]
        subprocess.run(gs_command, check=True)
        image_files = sorted([os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith(".png")])
        return image_files
    except subprocess.CalledProcessError as e:
        print(f"Ghostscript error: {e}")
        return []

def extract_text_from_image(image_path):
    reader = easyocr.Reader(['th', 'en'], gpu=True)
    image = Image.open(image_path)
    image_np = np.array(image)
    result = reader.readtext(image_np)
    sorted_data = sorted(result, key=lambda x: x[0][0][1])
    plain_text = "\n".join([text for _, text, _ in sorted_data])
    return plain_text

def chunk_text(text, tokenizer, max_tokens=200):
    tokens = tokenizer.tokenize(text)
    total_tokens = len(tokens)
    chunks = []
    start = 0
    while start < total_tokens:
        end = min(start + max_tokens, total_tokens)
        chunk = tokenizer.convert_tokens_to_string(tokens[start:end])
        chunks.append(chunk)

        start += max_tokens

    return chunks

def tag_and_clean_text(text, tagger, tokenizer, unwanted_pattern, max_tokens=200):
    text_chunks = chunk_text(text, tokenizer, max_tokens=max_tokens)
    tagged_text = []
    cleaned_text = []

    for i, chunk in enumerate(text_chunks):
        ner = tagger.get_ner(chunk, tag=True)
        if not ner:
            ner = chunk

        # Clean the tags
        pattern = r'<(?!ORGANIZATION|PERCENT|TIME)[^>]+>[^<]*?</[^>]+>'
        cleaned_ner = re.sub(pattern, '', ner)
        cleaned_ner = re.sub(r'</?(ORGANIZATION|PERCENT|TIME)>', '', cleaned_ner)
        cleaned_ner = re.sub(unwanted_pattern, '', cleaned_ner, flags=re.IGNORECASE)
        cleaned_ner = re.sub(r'\bal\b', 'ai', cleaned_ner, flags=re.IGNORECASE)
        # Append the tagged and cleaned text
        tagged_text.append(ner)
        cleaned_text.append(cleaned_ner)
    # Combine results from all chunks
    combined_tagged_text = "\n".join(tagged_text).strip()
    combined_cleaned_text = "\n".join(cleaned_text).strip()

    return combined_tagged_text, combined_cleaned_text


# Function to generate an answer using the prompt
def generate_answer(resume, job_description):
    formatted_prompt = resume_prompt.replace("{resume}", resume).replace("{jobdescribe}", job_description)
    truncated_prompt = formatted_prompt[:model_params["max_new_tokens"]]
    client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
    response = client.text_generation(formatted_prompt, **model_params)
    output = "".join(response)
    return output

# Extract classification from the result text
def extract_classification(text):
    pattern = r'\*\*\s*Classification\s*\**:\s*(?:\[)?\s*(Pass|Reconsider|Not Pass|Unrelated)\s*(?:\])?'
    match = re.search(pattern, text, re.IGNORECASE)
    if not match:
        pattern = r'Classification\s*:\s*(?:\[)?\s*(Pass|Reconsider|Not Pass|Unrelated)\s*(?:\])?'
        match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(1)
    return text

def process_file(file, job_description, is_pdf=True):
    if is_pdf:
        images = convert_pdf_to_images(file.name)
        raw_text = ""
        for image_path in images:
            raw_text += extract_text_from_image(image_path) + "\n"
    else:
        raw_text = extract_text_from_image(file.name)

    tagger = NamedEntityTagger()
    tokenizer = tagger.tokenizer

    unwanted_terms = [
        'ที่อยู่', 'โทรศัพท์', 'อีเมล', 'linkedin', ':', ',', '-', '|',
        'ประวัติส่วนตัว', 'เกี่ยวกับฉัน', 'about me', 'ชื่อ', 'สกุล', 'tell', 'โทร', 'โทรงาน',
        'ชื่อเล่น', 'อายุ', 'วันเกิด', 'พุทธ', 'ศาสนา', 'สัญชาติ', 'phone',
        'ช่องทางการติดต่อ', '_', 're sume', 'resume', 'resu me', 'birth', 'date',
        'address', 'email.', 'ประวัติ'
    ]
    unwanted_pattern = '|'.join(map(re.escape, unwanted_terms))

    tagged_text, cleaned_text = tag_and_clean_text(raw_text, tagger, tokenizer, unwanted_pattern)
    evaluation = generate_answer(cleaned_text, job_description)
    result = extract_classification(evaluation)

    df = pd.DataFrame([{
        "File": os.path.basename(file.name),
        "Raw_Text": raw_text,
        "Tagged_Text": tagged_text,
        "Cleaned_Text": cleaned_text,
        "Job_Description": job_description,
        "Evaluation": evaluation,
        "Result": result
    }])
    return df

# Function to process multiple files
def process_multiple_files(files, job_description, is_pdf=True):
    all_results = pd.DataFrame()
    for file in files:
        df = process_file(file, job_description, is_pdf)
        all_results = pd.concat([all_results, df], ignore_index=True)
    return all_results

# Function to save DataFrame to CSV
def save_dataframe(df):
    try:
        csv_path = '/content/output.csv'
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        return csv_path
    except Exception as e:
        print(f"Error saving DataFrame: {e}")
        return ""

# Function to process a batch of DataFrame rows
def generate_prompt_answer_optimized(row):
    resume = row.get('Cleaned_Text', "")
    job_description = row.get('Job_Description', "")
    result = generate_answer(resume, job_description)
    return result

# Function to process a batch of rows
def process_batch(batch_df):
    return [generate_prompt_answer_optimized(row) for _, row in batch_df.iterrows()]

# Batch processing with threading
def batch_process(df, batch_size=32):
    results = [None] * len(df)
    with ThreadPoolExecutor() as executor:
        futures = {}
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i + batch_size]
            future = executor.submit(process_batch, batch_df)
            futures[future] = (i, i + batch_size)

        for future in as_completed(futures):
            start_idx, end_idx = futures[future]
            batch_results = future.result()
            results[start_idx:end_idx] = batch_results

    return results

# Gradio Interface Logic
def gradio_interface(files, job_description, is_pdf, max_new_tokens, temperature, top_p, repetition_penalty):
    try:
        # Update model parameters
        model_params.update({
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,
            "top_p": top_p,
            "repetition_penalty": repetition_penalty
        })

        # Process files and pass job description
        df = process_multiple_files(files, job_description, is_pdf)
        df['Evaluation'] = batch_process(df, batch_size=32)
        df['Result'] = df['Evaluation'].apply(extract_classification)

        # Select only the required columns for output
        df_output = df[["Result", "Evaluation"]]

        csv_path = save_dataframe(df)
        return df_output, csv_path
    except Exception as e:
        print(f"Error in gradio_interface: {e}")
        return pd.DataFrame(), ""

# Define the Gradio interface
def create_gradio_interface():
    with gr.Blocks() as demo:
        with gr.Row():
            # Left Column for File Input
            with gr.Column(scale=1):
                file_input = gr.Files(label="Upload Files", file_count="multiple")
                is_pdf_input = gr.Checkbox(label="PDF", value=True)

            # Right Column for Parameters and Job Description
            with gr.Column(scale=1):
                job_input = gr.Textbox(placeholder="Enter the job description here...", label="Job Description", lines=10)

                # Model parameters
                max_new_tokens_input = gr.Slider(label="Max New Tokens", minimum=50, maximum=1024, value=500, step=50)
                temperature_input = gr.Slider(label="Temperature", minimum=0.01, maximum=1.0, value=0.1, step=0.1)
                top_p_input = gr.Slider(label="Top P", minimum=0.1, maximum=1.0, value=0.95, step=0.05)
                repetition_penalty_input = gr.Slider(label="Repetition Penalty", minimum=0.1, maximum=2.0, value=1.0, step=0.1)

                submit_btn = gr.Button("Process")

        # Center Bottom for Output
        with gr.Row():
            with gr.Column(scale=2):
                # Output DataFrame with limited rows visible and interactivity off
                output_df = gr.DataFrame(
                    headers=["Result", "Evaluation"],
                    type="pandas",
                    interactive=False
                )

                output_csv = gr.File(label="Download CSV")

        # Function to process inputs and generate output
        def on_submit(files, job_description, is_pdf, max_new_tokens, temperature, top_p, repetition_penalty):
            df, csv_path = gradio_interface(files, job_description, is_pdf, max_new_tokens, temperature, top_p, repetition_penalty)
            return df, csv_path

        # Bind the submit button to the processing function
        submit_btn.click(on_submit,
                         inputs=[file_input, job_input, is_pdf_input, max_new_tokens_input, temperature_input, top_p_input, repetition_penalty_input],
                         outputs=[output_df, output_csv])

    return demo

In [14]:
app = create_gradio_interface()
app.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://6bc7aafc7bf4f128e0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6bc7aafc7bf4f128e0.gradio.live


In [15]:
import pandas as pd
a = pd.read_csv('/content/output.csv')
a

,File,Raw_Text,Tagged_Text,Cleaned_Text,Job_Description,Evaluation,Result
0,resume dev1.pdf,thaaphoom babparn\n software engineer\npathum ...,<LOCATION>tha</LOCATION><ORGANIZATION>a</ORGAN...,aparn software engineer accenture application ...,**Software Engineer (Junior/Senior/Specialist)...,**Feedback**: Offer constructive feedback on t...,Pass
1,resume hr3.pdf,แอนนา แคทรีนา มาร์เคซี\nพนักงานฝ่ายบุคคล\n อาย...,<PERSON>แ</PERSON><PERSON>แอนนา แคทรีนา มาร์เค...,พนักงานฝ่ายบุคคล ข้อมูลโดยย่อ การศึกษา ระดับ ...,**Software Engineer (Junior/Senior/Specialist)...,**Feedback**: Offer constructive feedback on t...,Unrelated
2,resume ai8.pdf,alexander jones\nartificial intelligence engin...,<PERSON>alexander</PERSON> jones artificial in...,jones artificial inigence engineer innovative ...,**Software Engineer (Junior/Senior/Specialist)...,**Feedback**: Offer constructive feedback on t...,Reconsider
3,Resume ba5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\n nattamon. rad...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ มุ่งมันในการนําทักษะด้านการจัดกา...,**Software Engineer (Junior/Senior/Specialist)...,**Feedback**: Offer constructive feedback on t...,Unrelated


In [17]:
print(a['Cleaned_Text'].iloc[0])

aparn software engineer accenture application development senior analyst qjava
banking / fnancial project. suggested to team lead and take action to design. code and deploy statement digitization process with publish subscribe pattern with kafka to eliminate synchronous process cut processing time from 10 seconds to 2 seconds
manual process that takes to 3 minutes by hours for automated migration working by reading ajson confguration fle and calling the restful confgurations api. client credentials and authorization code methods for the exchange data process using implemented oauth 2.0 solution by kong api gateway (ce). for partner vendors with application by using jpa pagination api. optimized manual pagination logic on restfil api fom java spring boot improve the query time from 3400 milliseconds to 500 milliseconds. aware technology solutions.
coached new members of software engineers and instructed technologies to the team for innovative products. researched. designed and managed t

In [ ]:
## job = BA(https://th.jobsdb.com/Business-Analysis-jobs?jobId=78649722&type=standout) resume AI and BA
## job = BA(https://th.jobsdb.com/AI-Engineer-jobs?jobId=78442339&type=standout)) resume AI and BA

import pandas as pd
AI = pd.read_csv('/content/output.csv')
AI.to_csv('/content/drive/MyDrive/AIEngineer/resume/JOB/TEST_AI+BA.csv')
AI

In [ ]:
## job = BA(https://th.jobsdb.com/Business-Analysis-jobs?jobId=78649722&type=standout) resume AI and BA
## job = BA(https://th.jobsdb.com/AI-Engineer-jobs?jobId=78442339&type=standout)) resume AI and BA

import pandas as pd
AI = pd.read_csv('/content/output.csv')
AI.to_csv('/content/drive/MyDrive/AIEngineer/resume/JOB/TEST_AI+BA.csv')
AI

,File,Raw_Text,Tagged_Text,Cleaned_Text,Job_Description,Evaluation,Result
0,resume ai1.pdf,สมหญิง\n แก้วกาจญ์\n วิศวกรปัญญาประดิษฐ์\nal ท...,<PERSON>ส</PERSON><PERSON>สม</PERSON><PERSON>ห...,วิศวกรปัญญาประดิษฐ์ ai ที่มีประสบการณ์ 3 ปีในก...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Bor...,Borderline
1,resume ai2.pdf,วงศ์เจริญ\nณ็ชพล\n วิศวกรปัญญาประดิษฐ์\n ประว...,<PERSON>วงศ์เจริญ ณ็ชพล</PERSON> วิศวกรปัญญาปร...,วิศวกรปัญญาประดิษฐ์ ประวัติการศึกษา มหาวิทยาลั...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Bor...,Borderline
2,resume ai3.pdf,re sume\nประวัติส่วนตัว\n090-123-4567\n pimcha...,re sume ประวัติส่วนตัว<PHONE> 09</PHONE><PHONE...,ผู้ เชี่ยวชาญด้าน ai ที่มีประสบการณ์ 4 เขี ในด...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Not...,Not Pass
3,resume ai4.pdf,อาทิตย์\nรัตนวิจิตร\n วิศวกรปัญญาประดิษฐ์\nปร...,<PERSON>อ</PERSON><PERSON>อาทิตย์ รัตนวิจิตร</...,วิศวกรปัญญาประดิษฐ์ ประสบการณํ ทํางาน วิศวกร a...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Bor...,Borderline
4,resume ai5.pdf,ศิรินทิพย์\nศรีวัฒนกิจ\n วิศวกรปัญญาประดิษฐ์\n...,<PERSON>ศ</PERSON><PERSON>ศิริ</PERSON><PERSON...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์ทํางาน วิศวกร a...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Bor...,Borderline
5,resume ai6.pdf,linkedin:\nนภัสสร วิวัฒนาวงศ์\nlinkedin.com i...,linkedin:<PERSON> </PERSON><PERSON>น</PERSON><...,in วิศวกรปัญญาประดิษฐ์ ประสบการณ์การทํางาน วิ...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Bor...,Borderline
6,resume ai7.pdf,ปัญญา วิริยะชัย\n วิศวกรปัญญาประดิษฐ์\nประสบกา...,<PERSON>ป</PERSON><PERSON>ปัญญา วิริยะชัย</PER...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์การฝึกงาน บัณฑิต...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Bor...,Borderline
7,resume ai8.pdf,alexander jones\nartificial intelligence engin...,alexander jones artificial intelligence engine...,alexander jones artificial inigence engineer i...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Bor...,Borderline
8,Resume ba1.pdf,"พิมพ์นารา วัฒนชัย\nที่อยู่: 789 ถนนรัชดาภิเษก,...",<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,st เป้าหมายในการทํางาน นําความเชี่ยวชาญด้านการ...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Not...,Not Pass
9,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit .m@do...,<PERSON>ศ</PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ เป้าหมายในการทํางาน flnancial an...,Job Summary:\nThe senior AI engineer position ...,### **Evaluation**:\n- **Classification**: Not...,Not Pass


In [ ]:
print(AI['Evaluation'].iloc[2])

### **Evaluation**:
- **Classification**: Not Pass
- **Rationale**: The candidate's resume does not align with the job description for the senior AI engineer position. The resume is written in Thai and appears to describe experience in AI and machine learning, but it does not provide clear evidence of the required qualifications. Specifically, the resume does not mention hands-on knowledge in AI, machine learning, deep learning, PyTorch, Python, NLP, or any of the specialized machine learning areas listed in the job description. The candidate's experience with scikit-learn and Keras is mentioned, but this does not necessarily meet the requirement for hands-on knowledge in the specified areas. Additionally, the resume does not provide evidence of proven experience in applying AI to practical technology solutions, knowledge of various deep learning model architectures, or experience with training deep learning models on large datasets. The candidate's programming skills in Python and C++

In [ ]:
print(BA['Evaluation'].iloc[11])

### **Evaluation**:
- **Classification**: Not Pass
- **Rationale**: The candidate's resume does not align with the job description for a business analyst role. The candidate's experience is primarily in business analysis within the IT sector, focusing on technology solutions and process improvements. However, the job description requires experience in market research, competitive analysis, and business modeling, which are not prominently featured in the resume. Additionally, the candidate's educational background in Economics does not directly match the preferred qualifications, which include Business Administration, Finance, or related fields. The resume lacks specific skills and experience in market analysis, financial modeling, and strategic planning that are critical for the role described. Therefore, the candidate does not meet the key requirements for the position.


In [ ]:
print(BA['Raw_Text'].iloc[11])

 จิตรลดา
086-234-5678
jitlada.s@domain. com
โสภาเสริม
321 ถนนอโศกมนตรี, กรุงเทพฯ 10120
linkedin .com/in /jitlada-s
 business analyst
เป้าหมายในการทำงาน
ต้องการนำทักษะด้านการวิเคราะห์และความรู้เชิงธุรกิจมาช่วยเพิ่มประสิทธิภาพและผลลัพธ์ในการดำเนินธุรกิจขององค์กร
ประสบการณ์การทำงาน
ปัจจุบัน
 บริษัท modern solutions
 business analyst
เมษายน 2019
 ทำงานร่วมกับทีมเทคโนโลยีสารสนเทศและผู้บริหารในการระบุและแก้ไขปัญหาทางธุรกิจ
วิเคราะห์ ข้อมูลลูกค้าและจัดทำรายงานเพื่อสนับสนุนการตัดสินใจ
 ปรับปรุงกระบวนการทำงานภายในองค์กรเพื่อเพิ่มประสิทธิภาพ
 junior business analyst
 บริษัท nextgen tech
มีนาคม 2019
 มกราคม 2016
.
รวบรวมและวิเคราะห์ข้อมูลเพื่อสนับสนุนการตัดสินใจของผู้บริหาร
ประสานงานกับทีมพัฒนาในการจัดทำและทดสอบระบบใหม่
การศึกษา
 ปริญญาตรี สาขาเศรษฐศาสตร์
มหาวิทยาลัยเชียงใหม่, เชียงใหม่
2011
2015
ทักษะ
การวิเคราะห์ธุรกิจ: brd, frd, gap analysis
การจัดการโครงการ: microsoft project, trello
การวิเคราะห์ข้อมูล: excel, power bi, sol
การสื่อสาร: การเขียนรายงาน, การนำเสนอ



In [ ]:
print(BA['Tagged_Text'].iloc[11])

<PERSON>จ</PERSON><PERSON>จิตร</PERSON><PERSON>ลด</PERSON><PERSON>า</PERSON><PHONE> 08</PHONE><PHONE>6-2</PHONE><PHONE>34-5678</PHONE><PERSON> </PERSON><PERSON>jit</PERSON><PERSON>lada</PERSON><EMAIL>.</EMAIL><EMAIL>s@domain. com</EMAIL><LOCATION> </LOCATION><PERSON>โสภ</PERSON><PERSON>า</PERSON>เสริม<LOCATION> 3</LOCATION><LOCATION>21</LOCATION><LOCATION> </LOCATION><LOCATION>ถนนอโศกมนตรี</LOCATION><LOCATION>,</LOCATION><LOCATION> </LOCATION><LOCATION>กรุงเทพฯ</LOCATION><ZIP> 10</ZIP><ZIP>120</ZIP><URL> linkedin .com/in /</URL><PERSON>jit</PERSON><PERSON>lada</PERSON><URL>-s</URL><ORGANIZATION> business</ORGANIZATION> analyst เป้าหมายในการทํางาน ต้องการนําทักษะด้านการวิเคราะห์และความรู้เชิงธุรกิจมาช่วยเพิ่มประสิทธิภาพและผลลัพธ์ในการดําเนินธุรกิจขององค์กร ประสบการณ์การทํางาน ปัจจุบัน<ORGANIZATION> บริษัท modern solutions business analyst</ORGANIZATION><DATE> </DATE><DATE>เมษายน 2019</DATE> ทํางานร่วมกับทีมเทคโนโลยีสารสนเทศและผู้บริหารในการระบุและแก้ไขปัญหาทางธุรกิจ วิเคราะห์ ข้อมูลลูกค

RULE
- AI ทั้งหมด ไม่ผ่าน แต่ BA ยังมี BORDERLINE
-

In [ ]:
print(AI['Evaluation'].iloc[2])

Job Description:
- Required: 4+ years of experience in AI development, proficiency in Python and C++, ability to handle large data sets, experience with machine learning frameworks (Scikit-learn, TensorFlow, Keras), knowledge of image processing (OpenCV, Pillow), and a bachelor’s degree in Computer Science or a related field.

Resume:
- The candidate has 4 years of experience in AI development, specifically in developing AI solutions for data analysis. They are proficient in Python and C++, and have experience with Scikit-learn and Keras. They have skills in handling large data sets and creating models that improve business decision-making. They have experience with image processing using OpenCV and Pillow. They have a background in machine learning and deep learning using TensorFlow and Keras. They have a current role as a data engineer, developing machine learning models for financial forecasting to improve strategic decision-making. They have collaborated with data analysis teams to

In [ ]:
import pandas as pd
AI = pd.read_csv('/content/output.csv')
AI

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text,Evaluation,Result
0,RESUME AI ENG.pdf,"narakorn vorakul\n18/4 pak nam, mmueang samut ...",<PERSON>nar</PERSON><PERSON>akorn vorakul</PER...,address email.v ai engineer ตําแหน่ง birth da...,Strengths: [List the candidate's strengths as ...,Borderline
1,RESUME AI TH.pdf,นางสาวนารากร วรกุล\nที่อยู่:\n18/4 ถ.ศรีสมุทร ...,<PERSON>น</PERSON><PERSON>นางสาว</PERSON><PERS...,ai engineer ตําแหน่ง แนะนําตัวเบื้องต้น ผู้ ส...,Skills: [List the key skills that the candidat...,Not Pass
2,resume ai1.pdf,สมหญิง\n แก้วกาจญ์\n วิศวกรปัญญาประดิษฐ์\nal ท...,<PERSON>ส</PERSON><PERSON>สม</PERSON><PERSON>ห...,วิศวกรปัญญาประดิษฐ์ ai ที่มีประสบการณ์ 3 ปีในก...,Additional Notes: [Provide any additional note...,Pass
3,resume ai2.pdf,วงศ์เจริญ\nณ็ชพล\n วิศวกรปัญญาประดิษฐ์\n ประว...,<PERSON>วงศ์เจริญ ณ็ชพล</PERSON> วิศวกรปัญญาปร...,วิศวกรปัญญาประดิษฐ์ ประวัติการศึกษา มหาวิทยาลั...,Qualifications: [Score / 10]\nSkills: [Score /...,Pass
4,resume ai3.pdf,re sume\nประวัติส่วนตัว\n090-123-4567\n pimcha...,re sume ประวัติส่วนตัว<PHONE> 09</PHONE><PHONE...,ผู้ เชี่ยวชาญด้าน ai ที่มีประสบการณ์ 4 เขี ในด...,Strengths: [List at least two strong qualifica...,Pass
5,resume ai4.pdf,อาทิตย์\nรัตนวิจิตร\n วิศวกรปัญญาประดิษฐ์\nปร...,<PERSON>อ</PERSON><PERSON>อาทิตย์ รัตนวิจิตร</...,วิศวกรปัญญาประดิษฐ์ ประสบการณํ ทํางาน วิศวกร a...,Skills & Experience: [Score out of 10]\nCultur...,Pass
6,resume ai5.pdf,ศิรินทิพย์\nศรีวัฒนกิจ\n วิศวกรปัญญาประดิษฐ์\n...,<PERSON>ศ</PERSON><PERSON>ศิริ</PERSON><PERSON...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์ทํางาน วิศวกร a...,Standout Qualifications: [List any qualificati...,Pass
7,resume ai6.pdf,linkedin:\nนภัสสร วิวัฒนาวงศ์\nlinkedin.com i...,linkedin:<PERSON> </PERSON><PERSON>น</PERSON><...,in วิศวกรปัญญาประดิษฐ์ ประสบการณ์การทํางาน วิ...,Skills Alignment: [Assess the alignment of the...,Pass
8,resume ai7.pdf,ปัญญา วิริยะชัย\n วิศวกรปัญญาประดิษฐ์\nประสบกา...,<PERSON>ป</PERSON><PERSON>ปัญญา วิริยะชัย</PER...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์การฝึกงาน บัณฑิต...,Strengths: [List the candidate's strengths tha...,Pass
9,resume ai8.pdf,alexander jones\nartificial intelligence engin...,alexander jones artificial intelligence engine...,alexander jones artificial inigence engineer i...,Qualifications: [Evaluate the match between th...,Pass


In [ ]:
print(AI['Tagged_Text'].iloc[0])

<PERSON>nar</PERSON><PERSON>akorn vorakul</PERSON><LOCATION> 18</LOCATION><LOCATION>/4 pak nam</LOCATION><LOCATION>,</LOCATION><LOCATION> mmueang</LOCATION><LOCATION> samut prakan</LOCATION><LOCATION> district</LOCATION><LOCATION>,</LOCATION><LOCATION> samut prakan</LOCATION><ZIP> 10</ZIP><ZIP>270</ZIP> address:<PHONE> 09</PHONE><PHONE>9</PHONE><PHONE>-827-8887</PHONE> phone: email:<PERSON> nar</PERSON><PERSON>akorn</PERSON>.v<EMAIL>@</EMAIL><EMAIL>kkumail.com</EMAIL> al engineer ตําแหน่ง: birth date:<DATE> april 23, 2002</DATE> summary enthusiastic and detail oriented recent graduate with a bachelor's degree in statistics and data science, looking for an appropriate position. demonstrates strong analytical skills, attention to detail, and proficiency in python. experienced in data cleaning, preprocessing, machine learning implementation, and web application development with python., gained through academic projects and internships. education may bachelor of science (statistics and dat

In [ ]:
import pandas as pd
BA = pd.read_csv('/content/output.csv')
BA

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text,Evaluation,Result
0,Resume BA 2.pdf,"พิมพ์นารา วัฒนชัย\nที่อยู่: 789 ถนนรัชดาภิเษก,...",<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,st เป้าหมายในการทํางาน นําความเชี่ยวชาญด้านการ...,Skills Assessment: \n- [Skill 1]: [Level of p...,Borderline
1,Resume ba 5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\n nattamon. rad...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ มุ่งมันในการนําทักษะด้านการจัดกา...,Skills & Qualifications: [Assess the skills an...,Pass
2,Resume BA 3.pdf,จิตรลดา\n086-234-5678\njitlada.s@domain. com\...,<PERSON>จ</PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม business analyst เป้าหมายในการทํางาน ต้อ...,Job Description Requirements: [List the key re...,Pass
3,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit .m@do...,<PERSON>ศ</PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ เป้าหมายในการทํางาน flnancial an...,### **Job Description:**\nWe are seeking an ex...,Pass
4,Resume baa.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทํางาน มุ่งมันที่จะใช้ความรู้และท...,Specific Skill Set: [List the specific skills ...,Pass
